[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Todti-I/python-hw-1/blob/main/Part1.ipynb)

# Подгружаем зависимости

In [ ]:
%pip install requests
%pip install pandas
%pip install openpyxl
%pip install sqlalchemy

# Получаем id всех объектов

In [2]:
import requests
import os

data_filename= 'data.csv'

if not os.path.exists(data_filename):
    ids = []
    while len(ids) % 100 == 0:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={len(ids)}&limit=100&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
        res = requests.get(url)
        ids.extend(map(lambda item: item.get('objId'), res.json().get('data').get('list')))
    len(ids)

In [3]:
import pandas as pd

if os.path.exists(data_filename):
  data_frame = pd.read_csv(data_filename)
else:
  data = []
  for id in ids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    res = requests.get(url)
    data.append(res.json().get('data'))
  data_frame = pd.json_normalize(data)
  data_frame.to_csv(data_filename, index=False)  
  
data_frame

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,12307,25590,78,г Санкт-Петербург,Морская набережная. Ваш дом у моря,16.0,16.0,934,2022-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7694,14679,78,г Санкт-Петербург,Цветной город,4.0,5.0,0,2025-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,45757,552405,78,"г Санкт-Петербург, улица Ремесленная, д. 17",NEVA RESIDENCE,6.0,10.0,384,2024-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12060,25305,78,г Санкт-Петербург,Цветной город,6.0,7.0,0,2023-09-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12064,25306,78,г Санкт-Петербург,Цветной город,25.0,25.0,644,2023-09-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10655,30391,59422,62,"г Рязань, улица Интернациональная","Жилой Комплекс ""Бирюзова Парк""",17.0,17.0,254,2026-06-30,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10656,7910,14943,77,"г Москва, район Басманный",Дом Chkalov,3.0,21.0,0,2022-12-31,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10657,44068,533393,64,"рп Ровное, ул Карла Маркса, д. 1Б",NaN,1.0,1.0,8,2022-09-30,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10658,43078,515133,50,"г Серпухов, д. 4","ЖК ""Ивановские дворики""",14.0,14.0,109,2023-06-30,Монолит,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Сохраняем dataframe в excel

In [4]:
data_frame.to_excel('data.xlsx')

# Сохраняем dataframe в pickle

In [5]:
data_frame.to_pickle('data.pkl')

# Сохраняем dataframe в БД

In [6]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
data_frame.to_sql('data', con=engine)
engine.execute('select * from data limit 1').fetchall()

[(0, 12307, 25590, 78, 'г Санкт-Петербург', 'Морская набережная. Ваш дом у моря', 16.0, 16.0, 934, '2022-12-31', 'Другое', 'Другое', 'Нет', 0, 42347.0, '78-001053', 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/257641a6-f0ed-4129-94c5-38e4d0db7ec5', '08-09-2022 16:56', 59.9328, 30.1922, 1, 0, 'Бизнес', '25-09-2022 01:27', "[{'objId': 12307, 'pdId': 25590, 'rpdId': 632304, 'devId': 239, 'ordNum': 1, 'objRenderPhotoSize': 442567, 'objRenderPhotoUrl': 'https://xn--80az8a.x ... (156 characters truncated) ... oNm': 'Морская_набережная__1_очередь.jpg', 'objReadyDesc': 'Строится', 'objBuildTypeShortDesc': 'Жилое', 'knFlg': 1, 'loadDttm': '25-09-2022 01:11'}]", 0, 'normal', 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/02332888-9624-4396-aa57-01188233b9e5', 1, 21072, 16.0, 934, 42347.0, 22, 0, 1, '30-12-2023 12:00', '2,7 - 3', 0, 2.0, 0.0, 0.0, 77.0, 0.0, 1, 1, 0.0, 14.0, 14.0, 42, 0.910953786572839, 147302.0, 'ООО ПИТЕРГРАД (ИНН: 7813240241)', 1.0, '7 / 10', '[]', 0, 'Piskalenko.Y

# Сохраняем все фотографии

In [7]:
import ast

def download_photo(url):
  name = url.split('/')[-1]
  res = requests.get(url)
  open(f'./images/{name}.jpg', 'wb').write(res.content)

if not os.path.exists('./images'):
    os.mkdir('./images')

# изображения скачиваем только для первых 2 объектов
for item in list(data_frame.itertuples())[0:2]:
  photo_object = ast.literal_eval(item.photoRenderDTO)
  if not photo_object: continue
  for p in photo_object: download_photo(p.get('objRenderPhotoUrl'))